In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token 

dataset_path = r"C:\Users\adity\Desktop\NLP\April 11\DATASET 2\WELFake_Dataset.csv"

news_dataset = pd.read_csv(dataset_path)

news_dataset = news_dataset.sample(frac=0.03).reset_index(drop=True)  

train_news, eval_news = train_test_split(news_dataset, test_size=0.2) 

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def prepare_dataset(news):
    encodings = tokenizer(news['text'].tolist(), return_tensors='pt', max_length=512, truncation=True, padding='max_length', add_special_tokens=True)
    labels = news['label'].tolist()
    return NewsDataset(encodings, labels)

train_dataset = prepare_dataset(train_news)
eval_dataset = prepare_dataset(eval_news)

model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()
trainer.evaluate()



C:\Users\adity\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\adity\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\adity\anaconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). P

Step,Training Loss
10,3.191300
20,2.547200
30,1.494900
40,0.665000
50,0.385300
60,0.242500
70,0.395900
80,0.337200
90,0.225600
100,1.018800


{'eval_loss': 0.12696796655654907,
 'eval_runtime': 699.6767,
 'eval_samples_per_second': 0.619,
 'eval_steps_per_second': 0.079,
 'epoch': 1.0}

In [3]:
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import torch

eval_loader = DataLoader(eval_dataset, batch_size=8, shuffle=False)  # Adjust batch_size as needed

model.eval() 
predictions, true_labels = [], []

with torch.no_grad():
    for batch in eval_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        pred_labels = torch.argmax(logits, dim=-1)
        predictions.extend(pred_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

predictions = np.array(predictions)
true_labels = np.array(true_labels)

accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

report = classification_report(true_labels, predictions, target_names=['Fake', 'True'], output_dict=False)  # Set output_dict=True for a dict output
print(report)


C:\Users\adity\AppData\Local\Temp\ipykernel_17064\320283675.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Accuracy: 0.98
Precision: 0.98
Recall: 0.99
F1 Score: 0.98
              precision    recall  f1-score   support

        Fake       0.99      0.98      0.98       224
        True       0.98      0.99      0.98       209

    accuracy                           0.98       433
   macro avg       0.98      0.98      0.98       433
weighted avg       0.98      0.98      0.98       433

